In [2]:
import os
import json
import torch
import torch.optim as optim
import numpy as np
import argparse
import sys
sys.path.append("/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling")
from utils import make_flow, probs 
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')  # ensure it works on clusters without displaying plots
from torch.autograd.functional import hessian
import mplhep as hep
from torchmin import minimize
import torch.nn.functional as F

# Set paths
trial_dir = "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Fit_Weights/results_no_norm_fit_weights_NF/N_100000_seeds_60_bootstraps_1_4_16_128_15"
out_dir = "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Coverage_Check/generated_samples"
os.makedirs(out_dir, exist_ok=True)

# Reference architecture (consistent across models)
arch_config_path = "/work/gbadarac/MonoJet_NPLM/MonoJet_NPLM_analysis/Uncertainty_Modeling/Train_Models/EstimationNF_gaussians_outputs/N_100000_seeds_60_bootstraps_1_4_16_128_15"
#------------------
# Load models and initial weights 
# ------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #sets device to GPU if available 
os.makedirs(trial_dir, exist_ok=True)

with open(os.path.join(arch_config_path, "architecture_config.json")) as f:
    config = json.load(f) 

f_i_file = os.path.join(trial_dir, "f_i_averaged.pth")
f_i_statedicts = torch.load(f_i_file, map_location=device) #list of state_dicts 


In [ ]:

# ------------------
# Reconstruct flows f_i
# ------------------
N_generated=500000

f_i_models = []
sampled_mean=[]
for state_dict in f_i_statedicts:
    #for state_dict, config in zip(f_i_statedicts, configs):
    flow = make_flow(
        num_layers=config["num_layers"],
        hidden_features=config["hidden_features"],
        num_bins=config["num_bins"],
        num_blocks=config["num_blocks"],
        tail_bound=config["tail_bound"]
    ).to(device) #recreate the flow architecture for each model f_i 
    flow.load_state_dict(state_dict) #load the corresponding params into each model 
    flow.eval() # Set to eval mode to avoid training-time behavior (e.g., dropout)
    #You want all models to produce consistent, deterministic outputs for likelihood evaluation 
    #f_i_models.append(flow)
    gen_data = flow.sample((N_generated)).detach().numpy()
    mean = np.mean(gen_data, axis=0)
    sampled_mean.append(mean)
    print('mean',mean)

sampled_mean = np.stack(sampled_mean)
print(sampled_mean.shape)
np.save(os.path.join(out_dir, "generated_sampled_mean_60_models_4_16_128_15.npy"), sampled_mean)


In [4]:
# ------------------
# Reconstruct flows f_i
# ------------------
N_generated=500000

f_i_models = []
sampled_mean=[]
i=0
for state_dict in f_i_statedicts:
    i+=1
    #for state_dict, config in zip(f_i_statedicts, configs):
    flow = make_flow(
        num_layers=config["num_layers"],
        hidden_features=config["hidden_features"],
        num_bins=config["num_bins"],
        num_blocks=config["num_blocks"],
        tail_bound=config["tail_bound"]
    ).to(device) #recreate the flow architecture for each model f_i 
    flow.load_state_dict(state_dict) #load the corresponding params into each model 
    flow.eval() # Set to eval mode to avoid training-time behavior (e.g., dropout)
    #You want all models to produce consistent, deterministic outputs for likelihood evaluation 
    #f_i_models.append(flow)
    gen_data = flow.sample((N_generated)).detach().numpy()
    np.save(os.path.join(out_dir, "generated_samples_%i_models_4_16_128_15.npy"%(i)), gen_data)
    break